#### Libraries

In [1]:
# from coe_dataproc.monitoria import Monitoria
# from coe_dataproc.decorator import dataproc
# from datetime import datetime, timedelta
# import logging
# import sys
# from pyspark.sql import SparkSession, SQLContext
# from pyspark.sql.functions import *
# import pyspark.sql.functions as F
# from pyspark.sql.functions import year as spark_year
# from pyspark.sql.types import *
# from pyspark import SparkContext, SparkConf
# import base64
# from google.cloud import storage, secretmanager, bigquery
# import unidecode
# import datetime
# from pyspark.sql.window import Window

from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.sql.functions import year as spark_year
from pyspark.sql.types import *
from pyspark import SparkContext, SparkConf
from pyspark.sql.window import Window

#### Spark session init

In [3]:
conf = SparkConf().setAppName('business_case').set("spark.sql.sources.partitionOverwriteMode", "dynamic") \
                                      .set("hive.exec.dynamic.partition", "true") \
                                      .set("spark.sql.debug.maxToStringFields", 1000) \
                                      .set("hive.exec.dynamic.partition.mode", "nonstrict") \
                                      .set("mapreduce.fileoutputcommitter.marksuccessfuljobs", "false") \
                                      .set("viewsEnabled","true")\
                                      .set("materializationExpirationTimeInMinutes","20") 
    
spark = SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()
sc=spark.sparkContext
sqlContext = SQLContext(sc)

In [4]:
spark

#### Load DataSets

In [20]:
### Load Dataframes STAR - Usado para Joins ###
sales_source = spark.read.option("delimiter", "\t")\
    .option("header", True)\
    .option("encoding", "UTF-16")\
    .csv("abi_bus_case1_beverage_sales_20210726.csv")

channel_source = spark.read.option("delimiter", ",")\
    .option("header", True)\
    .csv("abi_bus_case1_beverage_channel_group_20210726.csv")

In [21]:
sales_source.printSchema()

root
 |-- DATE: string (nullable = true)
 |-- CE_BRAND_FLVR: string (nullable = true)
 |-- BRAND_NM: string (nullable = true)
 |-- Btlr_Org_LVL_C_Desc: string (nullable = true)
 |-- CHNL_GROUP: string (nullable = true)
 |-- TRADE_CHNL_DESC: string (nullable = true)
 |-- PKG_CAT: string (nullable = true)
 |-- Pkg_Cat_Desc: string (nullable = true)
 |-- TSR_PCKG_NM: string (nullable = true)
 |-- $ Volume: string (nullable = true)
 |-- YEAR: string (nullable = true)
 |-- MONTH: string (nullable = true)
 |-- PERIOD: string (nullable = true)



In [22]:
channel_source.printSchema()

root
 |-- TRADE_CHNL_DESC: string (nullable = true)
 |-- TRADE_GROUP_DESC: string (nullable = true)
 |-- TRADE_TYPE_DESC: string (nullable = true)



In [23]:
sales_source.show()

+--------+-------------+-----------+-------------------+------------------+--------------------+-------+------------+-------------+--------+----+-----+------+
|    DATE|CE_BRAND_FLVR|   BRAND_NM|Btlr_Org_LVL_C_Desc|        CHNL_GROUP|     TRADE_CHNL_DESC|PKG_CAT|Pkg_Cat_Desc|  TSR_PCKG_NM|$ Volume|YEAR|MONTH|PERIOD|
+--------+-------------+-----------+-------------------+------------------+--------------------+-------+------------+-------------+--------+----+-----+------+
|       �|         NULL|       NULL|               NULL|              NULL|                NULL|   NULL|        NULL|         NULL|    NULL|NULL| NULL|  NULL|
|1/1/2006|         3440|      LEMON|             CANADA|           LEISURE|         SPORT VENUE|   N20O|   20Z/600ML|.591L NRP 24L|   22.48|2006|    1|    1�|
|       �|         NULL|       NULL|               NULL|              NULL|                NULL|   NULL|        NULL|         NULL|    NULL|NULL| NULL|  NULL|
|1/1/2006|         3440|      LEMON|          

In [31]:
sales_source.dropna().show(truncate=False)

+--------+-------------+-----------+-------------------+------------------+------------------------+-------+------------+---------------+--------+----+-----+------+
|DATE    |CE_BRAND_FLVR|BRAND_NM   |Btlr_Org_LVL_C_Desc|CHNL_GROUP        |TRADE_CHNL_DESC         |PKG_CAT|Pkg_Cat_Desc|TSR_PCKG_NM    |$ Volume|YEAR|MONTH|PERIOD|
+--------+-------------+-----------+-------------------+------------------+------------------------+-------+------------+---------------+--------+----+-----+------+
|1/1/2006|3440         | LEMON     |CANADA             |LEISURE           |SPORT VENUE             |N20O   |20Z/600ML   |.591L NRP 24L  |22.48   |2006|1    |1�    |
|1/1/2006|3440         | LEMON     |NORTHEAST          |SUPERS            |SUPERETTE               |N20O   |20Z/600ML   |20Z NRP 24L    |100     |2006|1    |1�    |
|1/1/2006|3554         | STRAWBERRY|SOUTHEAST          |WORKPLACE         |PLANT / OFFICE          |N20O   |20Z/600ML   |20Z NRP 24L    |66.14   |2006|1    |1�    |
|1/1/2006|

In [32]:
channel_source.show()

+--------------------+----------------+---------------+
|     TRADE_CHNL_DESC|TRADE_GROUP_DESC|TRADE_TYPE_DESC|
+--------------------+----------------+---------------+
|         SPORT VENUE|   ENTERTAINMENT|      ALCOHOLIC|
|           SUPERETTE|        SERVICES|            MIX|
|      PLANT / OFFICE|        SERVICES|            MIX|
|   MASS MERCHANDISER|         GROCERY|            MIX|
|LIQUOR/BEER/WINE/...|         GROCERY|      ALCOHOLIC|
|   CONVENIENCE STORE|        SERVICES|            MIX|
|QUICK SERVICE RES...|   ENTERTAINMENT|      ALCOHOLIC|
|         SUPERMARKET|         GROCERY|            MIX|
|           ALL OTHER|           OTHER|            MIX|
|OTHER EATING + DR...|        SERVICES|            MIX|
|          RESTAURANT|   ENTERTAINMENT|      ALCOHOLIC|
|  HYPER-MERCHANDISER|         GROCERY|            MIX|
|          DRUG STORE|         GROCERY|            MIX|
|      TRANSPORTATION|        SERVICES|  NON ALCOHOLIC|
|  MILITARY-COMMISARY|  GOV & MILITARY|         

In [33]:
sales = sales_source.join(channel_source, ["TRADE_CHNL_DESC"],"inner")
sales.show()

+--------------------+--------+-------------+-----------+-------------------+------------------+-------+------------+---------------+--------+----+-----+------+----------------+---------------+
|     TRADE_CHNL_DESC|    DATE|CE_BRAND_FLVR|   BRAND_NM|Btlr_Org_LVL_C_Desc|        CHNL_GROUP|PKG_CAT|Pkg_Cat_Desc|    TSR_PCKG_NM|$ Volume|YEAR|MONTH|PERIOD|TRADE_GROUP_DESC|TRADE_TYPE_DESC|
+--------------------+--------+-------------+-----------+-------------------+------------------+-------+------------+---------------+--------+----+-----+------+----------------+---------------+
|         SPORT VENUE|1/1/2006|         3440|      LEMON|             CANADA|           LEISURE|   N20O|   20Z/600ML|  .591L NRP 24L|   22.48|2006|    1|    1�|   ENTERTAINMENT|      ALCOHOLIC|
|           SUPERETTE|1/1/2006|         3440|      LEMON|          NORTHEAST|            SUPERS|   N20O|   20Z/600ML|    20Z NRP 24L|     100|2006|    1|    1�|        SERVICES|            MIX|
|      PLANT / OFFICE|1/1/2006

In [36]:
sales = sales.select("DATE", "CE_BRAND_FLVR", "BRAND_NM", "Btlr_Org_LVL_C_Desc", "CHNL_GROUP", "TRADE_CHNL_DESC", "TRADE_GROUP_DESC", "TRADE_TYPE_DESC", "PKG_CAT", "Pkg_Cat_Desc", "TSR_PCKG_NM", "$ Volume", "YEAR", "MONTH", "PERIOD")
sales.show()

+--------+-------------+-----------+-------------------+------------------+--------------------+----------------+---------------+-------+------------+---------------+--------+----+-----+------+
|    DATE|CE_BRAND_FLVR|   BRAND_NM|Btlr_Org_LVL_C_Desc|        CHNL_GROUP|     TRADE_CHNL_DESC|TRADE_GROUP_DESC|TRADE_TYPE_DESC|PKG_CAT|Pkg_Cat_Desc|    TSR_PCKG_NM|$ Volume|YEAR|MONTH|PERIOD|
+--------+-------------+-----------+-------------------+------------------+--------------------+----------------+---------------+-------+------------+---------------+--------+----+-----+------+
|1/1/2006|         3440|      LEMON|             CANADA|           LEISURE|         SPORT VENUE|   ENTERTAINMENT|      ALCOHOLIC|   N20O|   20Z/600ML|  .591L NRP 24L|   22.48|2006|    1|    1�|
|1/1/2006|         3440|      LEMON|          NORTHEAST|            SUPERS|           SUPERETTE|        SERVICES|            MIX|   N20O|   20Z/600ML|    20Z NRP 24L|     100|2006|    1|    1�|
|1/1/2006|         3554| STRAW

In [39]:
# Create temporary view
sales.createOrReplaceTempView("sales")
spark.sql("SELECT DISTINCT CE_BRAND_FLVR, BRAND_NM FROM sales GROUP BY CE_BRAND_FLVR, BRAND_NM").show()

+-------------+-----------+
|CE_BRAND_FLVR|   BRAND_NM|
+-------------+-----------+
|         3554| STRAWBERRY|
|         3441|  RASPBERRY|
|         3697|      GRAPE|
|         3440|      LEMON|
+-------------+-----------+



In [40]:
spark.sql("SELECT DISTINCT PKG_CAT, Pkg_Cat_Desc, TSR_PCKG_NM FROM sales GROUP BY PKG_CAT, Pkg_Cat_Desc, TSR_PCKG_NM").show()

+-------+------------+---------------+
|PKG_CAT|Pkg_Cat_Desc|    TSR_PCKG_NM|
+-------+------------+---------------+
|   N128|12Z/355M 8NR|   12Z NRP 8P F|
|   N56P|    500ML 6P|     .5L NRP 6P|
|   N56P|    500ML 6P|   .5L NRP 6P S|
|   N20O|   20Z/600ML|.591L NRP 24L *|
|   N20O|   20Z/600ML|  .591L NRP 24L|
|   N20O|   20Z/600ML|  20z NRP 24L S|
|   N56P|    500ML 6P|    .5L NRP 6P*|
|   N20O|   20Z/600ML|    20Z NRP 24L|
+-------+------------+---------------+



In [43]:
spark.sql("SELECT DISTINCT Btlr_Org_LVL_C_Desc FROM sales").show()

+-------------------+
|Btlr_Org_LVL_C_Desc|
+-------------------+
|               WEST|
|          SOUTHWEST|
|            MIDWEST|
|             CANADA|
|          SOUTHEAST|
|          NORTHEAST|
|        GREAT LAKES|
+-------------------+



In [46]:
spark.sql("SELECT DISTINCT TRADE_CHNL_DESC, CHNL_GROUP, TRADE_GROUP_DESC, TRADE_TYPE_DESC FROM sales GROUP BY CHNL_GROUP, TRADE_CHNL_DESC, TRADE_GROUP_DESC, TRADE_TYPE_DESC").show()

+------------------+--------------------+----------------+---------------+
|        CHNL_GROUP|     TRADE_CHNL_DESC|TRADE_GROUP_DESC|TRADE_TYPE_DESC|
+------------------+--------------------+----------------+---------------+
|OTHER SMALL STORES|LOCAL+TRADITIONAL...|         GROCERY|            MIX|
|             CLUBS| CASH + CARRY RETAIL|         GROCERY|            MIX|
|       RETAIL COLD|RETAIL SPECIALITY...|         GROCERY|      ALCOHOLIC|
|           LEISURE|         SPORT VENUE|   ENTERTAINMENT|      ALCOHOLIC|
|         EDUCATION|PRIMARY/SECONDARY...|        ACADEMIC|  NON ALCOHOLIC|
|         WHOLESALE|           WHOLESALE|         GROCERY|            MIX|
|         EDUCATION|  COLLEGE/UNIVERSITY|        ACADEMIC|  NON ALCOHOLIC|
|CONVENIENCE RETAIL|   CONVENIENCE STORE|        SERVICES|            MIX|
|OTHER SMALL STORES|           ALL OTHER|           OTHER|            MIX|
| MASS MERCHANDISER|   MASS MERCHANDISER|         GROCERY|            MIX|
|       RETAIL COLD|GENER